In [ ]:
import sys; sys.path.append('..')
import demosaicing as dm
import importlib; importlib.reload(dm)

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

In [ ]:
kodak_dir = "../data/kodak/"
kodak_images = list(map(lambda x: f'{kodak_dir}{x}', os.listdir(kodak_dir)))
kodak = list(map(lambda x: cv.imread(x, -1), kodak_images))
crop = slice(500,600),slice(300,400)
dm.show_images([kodak[17], kodak[17][crop]], layouts='12', bgr2rgb=True)

In [ ]:
interp = {
    'original': kodak[17].copy()
}

## Spatial downsampling

Spatially downsample according to bayer pattern 

$$
B \quad G
$$
$$
G \quad R
$$

In [ ]:
img = np.ones((100,100,3), dtype=np.uint8) + 254
dm.show_images(list(map(lambda i: dm.color_channel(img,i), [0,1,2])), layouts='13', bgr2rgb=True)

In [ ]:
img = np.ones((100,100,3), dtype=np.uint8) + 254
img = dm.bayer_downsample(img)
dm.show_images(list(map(lambda i: dm.color_channel(img,i), [0,1,2])), layouts='13', bgr2rgb=True)

In [ ]:
lh = dm.bayer_downsample(kodak[17])
lhc = lh[350:400,200:250,:]
dm.show_images([*list(map(lambda i: dm.color_channel(lhc,i), [0,1,2])), lhc], layouts='14', bgr2rgb=True)

# bilinear interpolation 

+ http://www.dmi.unict.it/~battiato/mm1112/Parte%207.3%20%20-%20Demosaicing.pdf

The interpolated green intensity $G_{i,j}$ is simply the average of its immediate neighbors

$$
G_{i,j} = \frac{1}{4} \left( G_{i-1,j} + G_{i+1,j} + G_{i,j-1} + G_{i,j+1} \right)
$$

Similarly for red/blue intensities. The interpolation proceeds independently within each channel.

We can efficiently implement this method by convolution with 3x3 kernels over each of 3 color channels

$$
K_G = \frac{1}{4} \begin{bmatrix} 0&1&0 \\ 1&4&1 \\ 0&1&0 \end{bmatrix}
\quad\quad
K_R = K_B = \frac{1}{4} \begin{bmatrix} 1&2&1 \\ 2&4&2 \\ 1&2&1 \end{bmatrix}
$$ 

In [ ]:
lh = dm.bayer_downsample(kodak[17])
interp["bilinear"] = dm.demosaic_bilinear(lh)
lhc = interp["bilinear"][(*crop,)]
dm.show_images([*list(map(lambda i: dm.color_channel(lhc,i), [0,1,2])),lhc], layouts='15', bgr2rgb=True)
dm.show_images([kodak[17][crop], lhc, kodak[17][crop]-lhc], descriptions=['original','bilinear', 'diff'], layouts='13', bgr2rgb=True)

## Performance Metrics

+ MSE 
+ PSNR https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio

In [ ]:
dm.psnr(kodak[17], interp['bilinear'])

## OpenCV's demosaic method 

- [reference](https://docs.opencv.org/3.1.0/d7/d1b/group__imgproc__misc.html#gga4e0972be5de079fed4e3a10e24ef5ef0a8945844ab075687f4d4196abe1ce0db4)
    - bilinear
    - variable number of gradients
    - edge-aware

In [ ]:
demosaic_opencv = [
    ('opencv_bilinear', cv.COLOR_BayerBG2BGR),
    ('opencv_vng', cv.COLOR_BayerBG2BGR_VNG),
    ('opencv_edge_aware', cv.COLOR_BayerBG2BGR_EA)
]
lh = dm.bayer_downsample(kodak[17])
lh = np.sum(lh, axis=2, dtype=np.uint8)
for k,code in demosaic_opencv:
    interp[k] = cv.cvtColor(lh, code)

for k,img in interp.items():
    psnr = dm.psnr(interp["original"], img)
    print(f'{k:<20}\t{psnr}')

desc, imgs = zip(*list(interp.items()))
dm.show_images([img[(*crop,)] for img in imgs], desc, layouts='15', bgr2rgb=True)

## check performance metric over kodak dataset

In [ ]:
demosaic_algos = [
    ('bilinear',            dm.demosaic_bilinear),
    ('opencv_bilinear',     lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR)),
    ('opencv_vng',          lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR_VNG)),
    ('opencv_edge_aware',   lambda x: cv.cvtColor(x, COLOR_BayerBG2BGR_EA))
]

history = {}

for algo, f in demosaic_algos:
    history[algo] = dm.validate_kodak(f)
    print(f"{algo:>20}\t{np.mean(history[algo]['psnr'])}")
    


# opencv_linear = dm.validate_kodak(lambda x: cv.cvtColor(x, cv.COLOR_BayerBG2BGR))
    g
# lh = dm.bayer_downsample(kodak[17])
# lh = np.sum(lh, axis=2, dtype=np.uint8)
# for k,code in demosaic_opencv:
#     interp[k] = cv.cvtColor(lh, code)

# for k,img in interp.items():
#     psnr = dm.psnr(interp["original"], img)
#     print(f'{k:<20}\t{psnr}')


In [ ]:
plt.hist(opencv_linear['psnrs'])